In [2]:
import pandas as pd
import numpy as np

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout
from tensorflow.keras.preprocessing.text import one_hot

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#To avoid encoding errors
import chardet
with open('/content/train.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

To avoid **field larger than field limit (131072)** issue we need increase `field_size_limit` and according there might be cases where few bad lines might be appended at end so handle them use `error_bad_lines=False`

In [4]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [5]:
df = pd.read_csv("/content/train.csv",engine="python",encoding='utf-8',error_bad_lines=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 7354: unexpected end of data


In [6]:
X = df.drop('label',axis=1)
Y = df['label']

In [7]:
voc_size = 5000 #no.of unique vocabulary words you want store for this model

Encoding with `one_hot`

In [14]:
messages=X.copy()
messages.reset_index(inplace=True)

### Dataset Preprocessing
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub("[^a-zA-Z]", " ", str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [9]:
sent_length=20 #it will make the all inputs to be equal before passing to NN
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [11]:
X_final=np.array(embedded_docs)
y_final=np.array(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
### Model Training
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
77/77 [==============================] - 12s 98ms/step - loss: 0.4928 - accuracy: 0.7488 - val_loss: 0.3491 - val_accuracy: 0.8492
Epoch 2/10
77/77 [==============================] - 4s 53ms/step - loss: 0.1783 - accuracy: 0.9332 - val_loss: 0.2008 - val_accuracy: 0.9147
Epoch 3/10
77/77 [==============================] - 2s 27ms/step - loss: 0.0957 - accuracy: 0.9657 - val_loss: 0.2148 - val_accuracy: 0.9126
Epoch 4/10
77/77 [==============================] - 2s 27ms/step - loss: 0.0489 - accuracy: 0.9842 - val_loss: 0.2664 - val_accuracy: 0.9147
Epoch 5/10
77/77 [==============================] - 2s 26ms/step - loss: 0.0203 - accuracy: 0.9957 - val_loss: 0.2882 - val_accuracy: 0.9126
Epoch 6/10
77/77 [==============================] - 2s 25ms/step - loss: 0.0099 - accuracy: 0.9986 - val_loss: 0.3781 - val_accuracy: 0.9073
Epoch 7/10
77/77 [==============================] - 2s 28ms/step - loss: 0.0054 - accuracy: 0.9994 - val_loss: 0.3782 - val_accuracy: 0.9089
Epoch 8/10
7

In [12]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
